# Modelo de Suavización Exponencial Simple (SES)

Este notebook implementa el método de Suavización Exponencial Simple (Simple Exponential Smoothing - SES) para predecir las ventas de dos productos, utilizando tres tipos de validación temporal:
- Walk-Forward Validation
- Rolling Window
- Expanding Window

Posteriormente, se utiliza Optimización Bayesiana para encontrar los mejores hiperparámetros y entrenar el modelo final para producción.


In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
import optuna
warnings.filterwarnings('ignore')


In [16]:
# Cargar el dataset
df = pd.read_csv('data-set.csv', index_col=0)
print("Dataset cargado:")
print(f"Forma del dataset: {df.shape}")
print(f"\nPrimeras filas:")
print(df.head())
print(f"\nÚltimas filas:")
print(df.tail())
print(f"\nInformación del dataset:")
print(df.info())


Dataset cargado:
Forma del dataset: (127, 2)

Primeras filas:
    producto1   producto2
1  500.000000  200.000000
2  497.400893  210.686220
3  478.605317  222.018584
4  486.454125  233.920990
5  479.695678  238.402098

Últimas filas:
      producto1   producto2
123  164.610771  629.293034
124  150.881839  637.099467
125  151.788470  653.155282
126  137.047639  672.528345
127  141.990873  676.058092

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
Index: 127 entries, 1 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   producto1  127 non-null    float64
 1   producto2  127 non-null    float64
dtypes: float64(2)
memory usage: 3.0 KB
None


## Función SES (Simple Exponential Smoothing)

Implementación del método de Suavización Exponencial Simple para realizar predicciones. En SES, se utiliza un parámetro de suavización alpha (α) que controla el peso dado a las observaciones más recientes.


In [17]:
def ses_forecast(data, alpha=None, forecast_horizon=1):
    """
    Calcula la Suavización Exponencial Simple y realiza predicciones.
    
    Parameters:
    -----------
    data : array-like
        Serie temporal de datos
    alpha : float, optional
        Parámetro de suavización (0 < alpha <= 1). Si es None, se optimiza automáticamente.
    forecast_horizon : int
        Número de períodos a predecir (por defecto 1)
    
    Returns:
    --------
    float or array
        Predicción(es) usando SES
    """
    if len(data) < 2:
        # Si no hay suficientes datos, usar el último valor disponible
        return data[-1] if len(data) > 0 else 0
    
    # Convertir a pandas Series para usar con statsmodels
    data_series = pd.Series(data)
    
    try:
        # Crear y ajustar el modelo SES
        # Simple Exponential Smoothing: trend=None, seasonal=None
        model = ExponentialSmoothing(
            data_series,
            trend=None,
            seasonal=None,
            initialization_method='estimated'
        )
        
        # Si alpha es None, dejar que el modelo lo optimice
        # Si alpha está especificado, usarlo
        if alpha is not None:
            fit = model.fit(smoothing_level=alpha, optimized=False)
        else:
            fit = model.fit(optimized=True)
        
        # Realizar predicción
        forecast = fit.forecast(steps=forecast_horizon)
        
        # Si solo se predice un paso, devolver un escalar
        if forecast_horizon == 1:
            # Manejar tanto Series como array
            if hasattr(forecast, 'iloc'):
                return float(forecast.iloc[0])
            else:
                return float(forecast[0])
        else:
            # Devolver array de predicciones
            if hasattr(forecast, 'values'):
                return forecast.values
            else:
                return np.array(forecast)
        
    except Exception as e:
        # En caso de error, usar el último valor como predicción
        print(f"Advertencia en SES: {e}")
        return data[-1]


## Función 1: Walk-Forward Validation
    
En Walk-Forward Validation, se entrena el modelo con datos hasta un punto específico y se valida con el siguiente punto. Luego se avanza un paso y se repite el proceso.


In [18]:
def walk_forward_validation(dataset, alpha=None, train_size_min=10):
    """
    Realiza validación Walk-Forward para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización para SES. Si es None, se optimiza en cada ventana.
    train_size_min : int
        Tamaño mínimo de entrenamiento antes de comenzar la validación
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Walk-Forward Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = ses_forecast(train_data, alpha=alpha, forecast_horizon=1)
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 2: Rolling Window Validation

En Rolling Window Validation, se mantiene un tamaño fijo de ventana de entrenamiento que se desplaza a lo largo del tiempo.


In [19]:
def rolling_window_validation(dataset, alpha=None, train_window_size=20):
    """
    Realiza validación Rolling Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización para SES. Si es None, se optimiza en cada ventana.
    train_window_size : int
        Tamaño fijo de la ventana de entrenamiento
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Rolling Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar desde train_window_size hasta n-1
        for i in range(train_window_size, n):
            # Datos de entrenamiento: ventana fija de tamaño train_window_size
            train_data = data[i - train_window_size:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = ses_forecast(train_data, alpha=alpha, forecast_horizon=1)
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_window_size + 1,
                'ventana': f"{i - train_window_size}-{i}",
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_window_size + 1) % 20 == 0:
                print(f"  Iteración {i - train_window_size + 1}/{n - train_window_size}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Función 3: Expanding Window Validation

En Expanding Window Validation, la ventana de entrenamiento crece con cada iteración, comenzando desde un tamaño mínimo y expandiéndose hasta incluir todos los datos disponibles hasta ese punto.


In [20]:
def expanding_window_validation(dataset, alpha=None, train_size_min=10):
    """
    Realiza validación Expanding Window para cada producto del dataset.
    
    Parameters:
    -----------
    dataset : pandas.DataFrame
        Dataset con las columnas de productos
    alpha : float, optional
        Parámetro de suavización para SES. Si es None, se optimiza en cada ventana.
    train_size_min : int
        Tamaño mínimo inicial de la ventana de entrenamiento
    
    Returns:
    --------
    dict
        Diccionario con métricas para cada producto:
        - 'producto1': lista de diccionarios con métricas por iteración
        - 'producto2': lista de diccionarios con métricas por iteración
    """
    results = {}
    
    for producto in dataset.columns:
        print(f"\n=== Expanding Window Validation para {producto} ===")
        data = dataset[producto].values
        n = len(data)
        
        metrics_list = []
        
        # Comenzar validación desde train_size_min hasta n-1
        for i in range(train_size_min, n):
            # Datos de entrenamiento: desde el inicio hasta i (ventana que se expande)
            train_data = data[:i]
            # Dato de validación: i
            actual = data[i]
            
            # Realizar predicción
            try:
                prediction = ses_forecast(train_data, alpha=alpha, forecast_horizon=1)
            except Exception as e:
                print(f"Error en iteración {i}: {e}")
                prediction = train_data[-1]  # Fallback al último valor
            
            # Calcular métricas
            rmse = np.sqrt(mean_squared_error([actual], [prediction]))
            mae = mean_absolute_error([actual], [prediction])
            
            metrics_list.append({
                'iteracion': i - train_size_min + 1,
                'ventana': i,  # Tamaño de la ventana que se expande
                'actual': actual,
                'prediccion': prediction,
                'rmse': rmse,
                'mae': mae
            })
            
            if (i - train_size_min + 1) % 20 == 0:
                print(f"  Iteración {i - train_size_min + 1}/{n - train_size_min}: RMSE={rmse:.4f}, MAE={mae:.4f}")
        
        results[producto] = metrics_list
        print(f"  Total de iteraciones: {len(metrics_list)}")
    
    return results


## Ejecución de las Validaciones

Después de definir las funciones que permitirán ejecutar el modelo teniendo en cuenta los tres tipos de validación, ahora ejecutamos las tres validaciones para cada producto y calculamos las métricas promedio.


In [21]:
# Parámetros de validación
TRAIN_SIZE_MIN = 12  # Tamaño mínimo de entrenamiento
TRAIN_WINDOW_SIZE = 12  # Tamaño fijo para Rolling Window

# Ejecutar Walk-Forward Validation
print("=" * 60)
print("WALK-FORWARD VALIDATION")
print("=" * 60)
results_wf = walk_forward_validation(df, alpha=None, train_size_min=TRAIN_SIZE_MIN)


WALK-FORWARD VALIDATION

=== Walk-Forward Validation para producto1 ===
  Iteración 20/115: RMSE=3.6876, MAE=3.6876
  Iteración 40/115: RMSE=0.2880, MAE=0.2880
  Iteración 60/115: RMSE=1.9309, MAE=1.9309
  Iteración 80/115: RMSE=1.9821, MAE=1.9821
  Iteración 100/115: RMSE=23.3026, MAE=23.3026
  Total de iteraciones: 115

=== Walk-Forward Validation para producto2 ===
  Iteración 20/115: RMSE=21.1823, MAE=21.1823
  Iteración 40/115: RMSE=18.5389, MAE=18.5389
  Iteración 60/115: RMSE=15.0041, MAE=15.0041
  Iteración 80/115: RMSE=25.2706, MAE=25.2706
  Iteración 100/115: RMSE=14.5660, MAE=14.5660
  Total de iteraciones: 115


In [22]:
# Ejecutar Rolling Window Validation
print("\n" + "=" * 60)
print("ROLLING WINDOW VALIDATION")
print("=" * 60)
results_rw = rolling_window_validation(df, alpha=None, train_window_size=TRAIN_WINDOW_SIZE)



ROLLING WINDOW VALIDATION

=== Rolling Window Validation para producto1 ===
  Iteración 20/115: RMSE=4.3955, MAE=4.3955
  Iteración 40/115: RMSE=1.1571, MAE=1.1571
  Iteración 60/115: RMSE=1.9309, MAE=1.9309
  Iteración 80/115: RMSE=1.9821, MAE=1.9821
  Iteración 100/115: RMSE=23.2559, MAE=23.2559
  Total de iteraciones: 115

=== Rolling Window Validation para producto2 ===
  Iteración 20/115: RMSE=21.1823, MAE=21.1823
  Iteración 40/115: RMSE=18.2109, MAE=18.2109
  Iteración 60/115: RMSE=13.0698, MAE=13.0698
  Iteración 80/115: RMSE=30.9878, MAE=30.9878
  Iteración 100/115: RMSE=11.1658, MAE=11.1658
  Total de iteraciones: 115


In [23]:
# Ejecutar Expanding Window Validation
print("\n" + "=" * 60)
print("EXPANDING WINDOW VALIDATION")
print("=" * 60)
results_ew = expanding_window_validation(df, alpha=None, train_size_min=TRAIN_SIZE_MIN)



EXPANDING WINDOW VALIDATION

=== Expanding Window Validation para producto1 ===
  Iteración 20/115: RMSE=3.6876, MAE=3.6876
  Iteración 40/115: RMSE=0.2880, MAE=0.2880
  Iteración 60/115: RMSE=1.9309, MAE=1.9309
  Iteración 80/115: RMSE=1.9821, MAE=1.9821
  Iteración 100/115: RMSE=23.3026, MAE=23.3026
  Total de iteraciones: 115

=== Expanding Window Validation para producto2 ===
  Iteración 20/115: RMSE=21.1823, MAE=21.1823
  Iteración 40/115: RMSE=18.5389, MAE=18.5389
  Iteración 60/115: RMSE=15.0041, MAE=15.0041
  Iteración 80/115: RMSE=25.2706, MAE=25.2706
  Iteración 100/115: RMSE=14.5660, MAE=14.5660
  Total de iteraciones: 115


In [24]:
def calcular_metricas_promedio(results, nombre_validacion):
    """
    Calcula las métricas promedio para cada producto.
    
    Parameters:
    -----------
    results : dict
        Resultados de la validación
    nombre_validacion : str
        Nombre del tipo de validación
    
    Returns:
    --------
    dict
        Diccionario con métricas promedio por producto
    """
    metricas_promedio = {}
    
    print(f"\n{'='*60}")
    print(f"MÉTRICAS PROMEDIO - {nombre_validacion.upper()}")
    print(f"{'='*60}")
    
    for producto in results.keys():
        metrics_list = results[producto]
        
        # Calcular promedios
        rmse_promedio = np.mean([m['rmse'] for m in metrics_list])
        mae_promedio = np.mean([m['mae'] for m in metrics_list])
        
        # Calcular RMSE global (sobre todas las predicciones)
        actuales = [m['actual'] for m in metrics_list]
        predicciones = [m['prediccion'] for m in metrics_list]
        rmse_global = np.sqrt(mean_squared_error(actuales, predicciones))
        mae_global = mean_absolute_error(actuales, predicciones)
        
        metricas_promedio[producto] = {
            'rmse_promedio': rmse_promedio,
            'mae_promedio': mae_promedio,
            'rmse_global': rmse_global,
            'mae_global': mae_global,
            'num_iteraciones': len(metrics_list)
        }
        
        print(f"\n{producto}:")
        print(f"  RMSE Promedio (por iteración): {rmse_promedio:.4f}")
        print(f"  MAE Promedio (por iteración): {mae_promedio:.4f}")
        print(f"  RMSE Global (sobre todas las predicciones): {rmse_global:.4f}")
        print(f"  MAE Global (sobre todas las predicciones): {mae_global:.4f}")
        print(f"  Número de iteraciones: {len(metrics_list)}")
    
    return metricas_promedio

# Calcular métricas promedio para cada tipo de validación
metricas_wf = calcular_metricas_promedio(results_wf, "Walk-Forward")
metricas_rw = calcular_metricas_promedio(results_rw, "Rolling Window")
metricas_ew = calcular_metricas_promedio(results_ew, "Expanding Window")



MÉTRICAS PROMEDIO - WALK-FORWARD

producto1:
  RMSE Promedio (por iteración): 8.0068
  MAE Promedio (por iteración): 8.0068
  RMSE Global (sobre todas las predicciones): 10.0343
  MAE Global (sobre todas las predicciones): 8.0068
  Número de iteraciones: 115

producto2:
  RMSE Promedio (por iteración): 13.9157
  MAE Promedio (por iteración): 13.9157
  RMSE Global (sobre todas las predicciones): 16.2964
  MAE Global (sobre todas las predicciones): 13.9157
  Número de iteraciones: 115

MÉTRICAS PROMEDIO - ROLLING WINDOW

producto1:
  RMSE Promedio (por iteración): 8.3434
  MAE Promedio (por iteración): 8.3434
  RMSE Global (sobre todas las predicciones): 10.2999
  MAE Global (sobre todas las predicciones): 8.3434
  Número de iteraciones: 115

producto2:
  RMSE Promedio (por iteración): 13.9478
  MAE Promedio (por iteración): 13.9478
  RMSE Global (sobre todas las predicciones): 16.6496
  MAE Global (sobre todas las predicciones): 13.9478
  Número de iteraciones: 115

MÉTRICAS PROMEDIO -

## Comparación de Resultados y Selección del Mejor Método de Validación

Comparamos los RMSE promedio de cada tipo de validación para seleccionar el mejor método.


In [25]:
# Crear DataFrame comparativo
comparacion = []

for producto in df.columns:
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Walk-Forward',
        'RMSE_Promedio': metricas_wf[producto]['rmse_promedio'],
        'RMSE_Global': metricas_wf[producto]['rmse_global'],
        'MAE_Promedio': metricas_wf[producto]['mae_promedio'],
        'MAE_Global': metricas_wf[producto]['mae_global']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Rolling Window',
        'RMSE_Promedio': metricas_rw[producto]['rmse_promedio'],
        'RMSE_Global': metricas_rw[producto]['rmse_global'],
        'MAE_Promedio': metricas_rw[producto]['mae_promedio'],
        'MAE_Global': metricas_rw[producto]['mae_global']
    })
    comparacion.append({
        'Producto': producto,
        'Validacion': 'Expanding Window',
        'RMSE_Promedio': metricas_ew[producto]['rmse_promedio'],
        'RMSE_Global': metricas_ew[producto]['rmse_global'],
        'MAE_Promedio': metricas_ew[producto]['mae_promedio'],
        'MAE_Global': metricas_ew[producto]['mae_global']
    })

df_comparacion = pd.DataFrame(comparacion)
print("\n" + "=" * 80)
print("COMPARACIÓN DE MÉTODOS DE VALIDACIÓN")
print("=" * 80)
print(df_comparacion.to_string(index=False))

# Identificar el mejor método para cada producto
print("\n" + "=" * 80)
print("MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)")
print("=" * 80)
for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor = producto_df.loc[producto_df['RMSE_Promedio'].idxmin()]
    print(f"\n{producto}:")
    print(f"  Mejor método: {mejor['Validacion']}")
    print(f"  RMSE Promedio: {mejor['RMSE_Promedio']:.4f}")
    print(f"  RMSE Global: {mejor['RMSE_Global']:.4f}")



COMPARACIÓN DE MÉTODOS DE VALIDACIÓN
 Producto       Validacion  RMSE_Promedio  RMSE_Global  MAE_Promedio  MAE_Global
producto1     Walk-Forward       8.006807    10.034318      8.006807    8.006807
producto1   Rolling Window       8.343419    10.299911      8.343419    8.343419
producto1 Expanding Window       8.006807    10.034318      8.006807    8.006807
producto2     Walk-Forward      13.915671    16.296403     13.915671   13.915671
producto2   Rolling Window      13.947772    16.649630     13.947772   13.947772
producto2 Expanding Window      13.915671    16.296403     13.915671   13.915671

MEJOR MÉTODO DE VALIDACIÓN POR PRODUCTO (basado en RMSE Promedio)

producto1:
  Mejor método: Expanding Window
  RMSE Promedio: 8.0068
  RMSE Global: 10.0343

producto2:
  Mejor método: Walk-Forward
  RMSE Promedio: 13.9157
  RMSE Global: 16.2964


## Optimización Bayesiana para Encontrar el Mejor Alpha

Utilizamos Optimización Bayesiana (Optuna) para encontrar el mejor parámetro alpha para cada producto, usando el mejor método de validación identificado.


In [26]:
def objective_walk_forward(trial, data, train_size_min=10):
    """
    Función objetivo para optimización bayesiana usando Walk-Forward Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = ses_forecast(train_data, alpha=alpha, forecast_horizon=1)
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_rolling_window(trial, data, train_window_size=20):
    """
    Función objetivo para optimización bayesiana usando Rolling Window Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_window_size, n):
        train_data = data[i - train_window_size:i]
        actual = data[i]
        
        try:
            prediction = ses_forecast(train_data, alpha=alpha, forecast_horizon=1)
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)

def objective_expanding_window(trial, data, train_size_min=10):
    """
    Función objetivo para optimización bayesiana usando Expanding Window Validation.
    """
    alpha = trial.suggest_float('alpha', 0.01, 0.99, log=False)
    
    n = len(data)
    errors = []
    
    for i in range(train_size_min, n):
        train_data = data[:i]
        actual = data[i]
        
        try:
            prediction = ses_forecast(train_data, alpha=alpha, forecast_horizon=1)
            error = np.sqrt(mean_squared_error([actual], [prediction]))
            if np.isfinite(error):
                errors.append(error)
        except:
            continue
    
    if len(errors) == 0:
        return float('inf')
    
    return np.mean(errors)


In [27]:
# Determinar el mejor método de validación para cada producto
mejor_metodo_por_producto = {}

for producto in df.columns:
    producto_df = df_comparacion[df_comparacion['Producto'] == producto]
    mejor_idx = producto_df['RMSE_Promedio'].idxmin()
    mejor_metodo = producto_df.loc[mejor_idx, 'Validacion']
    mejor_metodo_por_producto[producto] = mejor_metodo
    print(f"{producto}: Mejor método = {mejor_metodo}")

# Realizar optimización bayesiana para cada producto usando su mejor método
mejores_params = {}

print("\n" + "=" * 80)
print("OPTIMIZACIÓN BAYESIANA")
print("=" * 80)

for producto in df.columns:
    print(f"\n--- Optimizando {producto} ---")
    data = df[producto].values
    mejor_metodo = mejor_metodo_por_producto[producto]
    
    # Crear función objetivo según el mejor método
    if mejor_metodo == 'Walk-Forward':
        objective_func = lambda trial: objective_walk_forward(trial, data, TRAIN_SIZE_MIN)
    elif mejor_metodo == 'Rolling Window':
        objective_func = lambda trial: objective_rolling_window(trial, data, TRAIN_WINDOW_SIZE)
    else:  # Expanding Window
        objective_func = lambda trial: objective_expanding_window(trial, data, TRAIN_SIZE_MIN)
    
    # Crear estudio de Optuna
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
    
    # Ejecutar optimización
    study.optimize(objective_func, n_trials=50, show_progress_bar=True)
    
    mejores_params[producto] = {
        'alpha': study.best_params['alpha'],
        'best_rmse': study.best_value,
        'metodo_validacion': mejor_metodo
    }
    
    print(f"  Mejor alpha: {study.best_params['alpha']:.4f}")
    print(f"  Mejor RMSE: {study.best_value:.4f}")
    print(f"  Método usado: {mejor_metodo}")

print("\n" + "=" * 80)
print("RESUMEN DE MEJORES PARÁMETROS")
print("=" * 80)
for producto, params in mejores_params.items():
    print(f"\n{producto}:")
    print(f"  Alpha óptimo: {params['alpha']:.4f}")
    print(f"  RMSE: {params['best_rmse']:.4f}")
    print(f"  Método de validación: {params['metodo_validacion']}")


[I 2025-11-30 13:05:48,362] A new study created in memory with name: no-name-5ecdc5e4-bb85-4f1d-8a16-a6ad11fafbbd


producto1: Mejor método = Expanding Window
producto2: Mejor método = Walk-Forward

OPTIMIZACIÓN BAYESIANA

--- Optimizando producto1 ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:05:48,715] Trial 0 finished with value: 12.516797550353052 and parameters: {'alpha': 0.4661793808994339}. Best is trial 0 with value: 12.516797550353052.
[I 2025-11-30 13:05:48,946] Trial 1 finished with value: 50.09598544217897 and parameters: {'alpha': 0.06258872655681176}. Best is trial 0 with value: 12.516797550353052.
[I 2025-11-30 13:05:49,166] Trial 2 finished with value: 17.524271378518637 and parameters: {'alpha': 0.30222436769441813}. Best is trial 0 with value: 12.516797550353052.
[I 2025-11-30 13:05:49,419] Trial 3 finished with value: 7.986167521581934 and parameters: {'alpha': 0.9248152910036475}. Best is trial 3 with value: 7.986167521581934.
[I 2025-11-30 13:05:49,725] Trial 4 finished with value: 13.404079204774503 and parameters: {'alpha': 0.4269109178655195}. Best is trial 3 with value: 7.986167521581934.
[I 2025-11-30 13:05:49,989] Trial 5 finished with value: 12.106772744414805 and parameters: {'alpha': 0.48687756398374565}. Best is trial 3 with va

[I 2025-11-30 13:05:57,631] A new study created in memory with name: no-name-da94db66-140e-48aa-95da-7f3a471f3b89


[I 2025-11-30 13:05:57,282] Trial 47 finished with value: 7.96179219302607 and parameters: {'alpha': 0.9362774865763811}. Best is trial 41 with value: 7.888681308937618.
[I 2025-11-30 13:05:57,457] Trial 48 finished with value: 9.142804752982537 and parameters: {'alpha': 0.7167704420658072}. Best is trial 41 with value: 7.888681308937618.
[I 2025-11-30 13:05:57,628] Trial 49 finished with value: 14.587543518005937 and parameters: {'alpha': 0.3828805337149942}. Best is trial 41 with value: 7.888681308937618.
  Mejor alpha: 0.9890
  Mejor RMSE: 7.8887
  Método usado: Expanding Window

--- Optimizando producto2 ---


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-11-30 13:05:57,810] Trial 0 finished with value: 19.08668514667183 and parameters: {'alpha': 0.415510591819895}. Best is trial 0 with value: 19.08668514667183.
[I 2025-11-30 13:05:57,986] Trial 1 finished with value: 20.586534569137267 and parameters: {'alpha': 0.3703991239669915}. Best is trial 0 with value: 19.08668514667183.
[I 2025-11-30 13:05:58,159] Trial 2 finished with value: 37.73980152786019 and parameters: {'alpha': 0.1619781278446422}. Best is trial 0 with value: 19.08668514667183.
[I 2025-11-30 13:05:58,332] Trial 3 finished with value: 18.82795848588279 and parameters: {'alpha': 0.4246824965777528}. Best is trial 3 with value: 18.82795848588279.
[I 2025-11-30 13:05:58,503] Trial 4 finished with value: 64.1433436306668 and parameters: {'alpha': 0.08503334909203324}. Best is trial 3 with value: 18.82795848588279.
[I 2025-11-30 13:05:58,802] Trial 5 finished with value: 27.976848803144456 and parameters: {'alpha': 0.24128303419872527}. Best is trial 3 with value: 18.

## Modelo Final para Producción

Entrenamos el modelo final con los mejores parámetros encontrados usando todos los datos disponibles y realizamos predicciones para producción.


In [28]:
# Entrenar modelos finales para cada producto con los mejores parámetros
modelos_finales = {}
predicciones_finales = {}

print("=" * 80)
print("ENTRENAMIENTO DE MODELOS FINALES PARA PRODUCCIÓN")
print("=" * 80)

for producto in df.columns:
    print(f"\n--- {producto} ---")
    data = df[producto].values
    alpha_optimo = mejores_params[producto]['alpha']
    
    # Entrenar modelo final con todos los datos
    data_series = pd.Series(data)
    model = ExponentialSmoothing(
        data_series,
        trend=None,
        seasonal=None,
        initialization_method='estimated'
    )
    fit = model.fit(smoothing_level=alpha_optimo, optimized=False)
    
    modelos_finales[producto] = fit
    
    # Realizar predicción para el próximos mes
    forecast_horizon = 1
    forecast = fit.forecast(steps=forecast_horizon)
    
    predicciones_finales[producto] = forecast
    
    print(f"  Alpha usado: {alpha_optimo:.4f}")
    print(f"  Último valor observado: {data[-1]:.4f}")
    print(f"  Predicciones para los próximos {forecast_horizon} períodos:")
    for i, pred in enumerate(forecast, 1):
        print(f"    Período {i}: {pred:.4f}")

# Crear DataFrame con las predicciones
df_predicciones = pd.DataFrame(predicciones_finales)
df_predicciones.index = [f'Período_{i+1}' for i in range(len(df_predicciones))]

print("\n" + "=" * 80)
print("PREDICCIONES FINALES PARA PRODUCCIÓN")
print("=" * 80)
print(df_predicciones)


ENTRENAMIENTO DE MODELOS FINALES PARA PRODUCCIÓN

--- producto1 ---
  Alpha usado: 0.9890
  Último valor observado: 141.9909
  Predicciones para los próximos 1 períodos:
    Período 1: 141.9381

--- producto2 ---
  Alpha usado: 0.8802
  Último valor observado: 676.0581
  Predicciones para los próximos 1 períodos:
    Período 1: 675.3278

PREDICCIONES FINALES PARA PRODUCCIÓN
            producto1   producto2
Período_1  141.938108  675.327802


## Resumen Final

### Resultados de Validación

- **Walk-Forward Validation**: Se evaluó el modelo avanzando paso a paso en el tiempo
- **Rolling Window Validation**: Se evaluó con una ventana fija que se desplaza
- **Expanding Window Validation**: Se evaluó con una ventana que crece progresivamente

### Optimización Bayesiana

Se utilizó Optimización Bayesiana (Optuna) para encontrar el mejor parámetro alpha para cada producto, usando el método de validación que obtuvo el menor RMSE promedio.

### Modelo Final

Los modelos finales fueron entrenados con todos los datos disponibles usando los mejores parámetros encontrados, y están listos para realizar predicciones en producción.
